In [3]:
import pandas as pd
import nltk
import unicodedata
import string
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
nltk.download('wordnet')
nltk.download('punkt')
from bs4 import BeautifulSoup

df = pd.read_json('Electronics_5.json', lines=True)

In [52]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0528881469,"[0, 0]",5,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1370131200
1,0528881469,"[12, 15]",1,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1290643200
2,0528881469,"[43, 45]",3,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1283990400
3,0528881469,"[9, 10]",2,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1290556800
4,0528881469,"[0, 0]",1,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1317254400


In [58]:
df_neg = df[df['overall'] == 1]
df_pos = df[df['overall'] == 5]

In [67]:
df_pos['reviewText'] = df_pos.reviewText.map(sent_tokenize)
#df_neg['reviewText'] = df_neg.reviewText.map(sent_tokenize)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


1          [I'm a professional OTR truck driver, and I bo...
4          [I've had mine for a year and here's what we g...
71         [Bought NEW but I could tell right away that m...
82         [I checked around Amazon as well as some other...
167        [I consider myself to be a good DIYer., I've i...
232        [If less money had been spent on packaging and...
233        [I bought this mount for our bedroom in order ...
236        [Mounting a 42 inch LG TV weighing 33 poundsPR...
242        [Have no idea why people love this thing so mu...
255        [Check your SN with Barnes & Noble's 800 numbe...
260        [If all you want is a tablet for reading B&N; ...
267        [The customer support people at Barnes & Noble...
268        [If you want an e-reader, get an e-reader - th...
273        [Update 2-Sep-13:Put on CM 10.2.2 (Android 4.2...
295        [Why pay to be restricted, I simply didn't see...
320        [Highly disappointed., I purchased the new B/W...
324        [I bought thi

In [74]:
pos = df_pos['reviewText'].values
neg = df_neg['reviewText'].values

In [79]:
raw_reviews = []
for i in range(min(len(neg), len(pos))):
    raw_reviews.append((pos[i], 1))
    raw_reviews.append((neg[i], 1))

(['We got this GPS for my husband who is an (OTR) over the road trucker.',
  'Very Impressed with the shipping time, it arrived a few days earlier than expected...  within a week of use however it started freezing up... could of just been a glitch in that unit.',
  'Worked great when it worked!',
  'Will work great for the normal person as well but does have the "trucker" option.',
  '(the big truck routes - tells you when a scale is coming up ect...)  Love the bigger screen, the ease of use, the ease of putting addresses into memory.',
  "Nothing really bad to say about the unit with the exception of it freezing which is probably one in a million and that's just my luck.",
  'I contacted the seller and within minutes of my email I received a email back with instructions for an exchange!',
  'VERY impressed all the way around!'],
 1)

In [70]:
# 클렌징
# stop = stopwords.words('english')
lmtzr = WordNetLemmatizer()


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_review(text: str) -> str:
    # 1. unicode to ASCII
    text = unicodeToAscii(text)
            
    # 2. Remove HTML
    text = BeautifulSoup(text, "lxml").get_text()
    
    # 3. Remove Punctuations
    text = "".join(v for v in text if v not in string.punctuation).lower()    

    return text

In [81]:
reviews = []
count = 0
  
for review in raw_reviews[:len(raw_reviews)//4]:
    text = review[0]
    code = review[1]
    for s in text:
        new_s = clean_review(s)
        if(len(new_s) > 10 and new_s[0].isalpha()):
            reviews.append((new_s, code))
    if(not count % 10000):
        print(count)
    count += 1

0
10000
20000


/usr/local/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://www.flickr.com/photos/rafquil/13345586614/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


30000
40000
50000


/usr/local/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'////'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [84]:
len(reviews)

310219

In [85]:
import pickle
import gzip

with gzip.open('pre_reviews_2.pickle', 'wb') as f:
    pickle.dump(reviews, f)